<a href="https://colab.research.google.com/github/vitthal-bhandari/incremental-semi-supervised-training/blob/master/Self%20training/Self_training_Task_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initializing environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Oct  9 14:27:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
# !pip install datasets
# !pip install transformers
!pip3 install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install wandb
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
     |██████████████████████████████▎ | 834.1 MB 101.1 MB/s eta 0:00:01tcmalloc: large alloc 1147494400 bytes == 0x39fc2000 @  0x7f90c663c615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████████████████████| 881.9 MB 2.0 kB/s 
     |████████████████████████████████| 23.2 MB 2.0 MB/s 
     |████████████████████████████████| 2.9 MB 2.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: to

In [4]:
API_KEY = 'ddb1a3a21db80aada85fe7d1e39a95d2abbdc426'

In [5]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import r_regression
import pandas as pd
import numpy as np
import torch

# import wandb
# wandb.login()

# Reading & splitting data

In [6]:
!git clone https://github.com/vitthal-bhandari/incremental-semi-supervised-training.git

Cloning into 'incremental-semi-supervised-training'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 38 (delta 1), reused 12 (delta 1), pack-reused 26
Unpacking objects: 100% (38/38), done.
Checking out files: 100% (19/19), done.


In [7]:
#Reading untouched data (non-preprocessed, non-augmented)

dataset = pd.read_csv("/content/incremental-semi-supervised-training/Data/starting kit/train_all_tasks.csv")
dataset.rename(columns={'label_sexist': 'labels'}, inplace=True)

print(len(dataset))

14000


In [27]:
#Reading untouched data (non-preprocessed, non-augmented)

dev_df = pd.read_csv("/content/incremental-semi-supervised-training/Data/dev_task_a_entries.csv")

print(len(dev_df))

2000


In [9]:
gab_data_df = pd.read_csv("/content/incremental-semi-supervised-training/Data/starting kit/gab_1M_unlabelled.csv")
reddit_data_df = pd.read_csv("/content/incremental-semi-supervised-training/Data/starting kit/reddit_1M_unlabelled.csv")

In [10]:
dataset.head()

,rewire_id,text,labels,label_category,label_vector
0,sexism2022_english-7358,"Damn, this writing was pretty chaotic",not sexist,none,none
1,sexism2022_english-2367,"Yeah, and apparently a bunch of misogynistic v...",not sexist,none,none
2,sexism2022_english-3073,How the FUCK is this woman still an MP!!!???,not sexist,none,none
3,sexism2022_english-14895,Understand. Know you're right. At same time I ...,not sexist,none,none
4,sexism2022_english-4118,Surprized they didn't stop and rape some women,not sexist,none,none


In [28]:
dev_df.head()

,rewire_id,text
0,sexism2022_english-11999,I do not try to hard. I do what guys who get w...
1,sexism2022_english-9895,That sucks. We elected a Navy Seal in Missouri...
2,sexism2022_english-11623,Walk away and never look back. If she reaches ...
3,sexism2022_english-19035,basically I placed more value on her than I sh...
4,sexism2022_english-895,"NYT and Asian ""part ways."" But why on earth wa..."


In [12]:
gab_data_df.head()

,text
0,JUST SAYING.... YOU LOSE BIATCHES
1,HOW WOULD I DESCRIBE BERKELEY RIOTS? 1) Charmi...
2,Or maybe a little south of Herring Run. More l...
3,That's funny...but
4,Unbelievable! It Was (((Rosenstein))) Who Hid ...


In [13]:

def split_stratified_into_train_val_test(df_input, stratify_colname='y',
                                         frac_train=0.7, frac_val=0.3,
                                         random_state=45):

    if frac_train + frac_val != 1.0:
        raise ValueError('fractions %f, %f do not add up to 1.0' % \
                         (frac_train, frac_val))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_val, y_train, y_val = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val)

    return df_train, df_val

In [14]:
df_train, df_val = \
    split_stratified_into_train_val_test(dataset, stratify_colname='labels', frac_train=0.80, frac_val=0.20)

In [15]:
df_train["labels"].value_counts()

not sexist    8482
sexist        2718
Name: labels, dtype: int64

In [16]:
df_val["labels"].value_counts()

not sexist    2120
sexist         680
Name: labels, dtype: int64

# Instantiating model

In [17]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
    
# result, model_outputs, wrong_predictions = model.eval_model(df_dev, f1=f1_multiclass, acc=accuracy_score)

# result

In [18]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [19]:
dataset = dataset[['text', 'labels']]

In [20]:
dataset.to_csv('dataset_1.csv', index=False)

In [21]:
def training_model(i, train_dataset):

  output_dir = "iteration_" + str(i)

  # define hyperparameter
  train_args = {
      "reprocess_input_data": True,
      "output_dir": output_dir,
      "overwrite_output_dir": True,
      "manual_seed": 4,
      "use_multiprocessing": True,
      "train_batch_size": 16,
      "labels_list": ["not sexist", "sexist"],
      "learning_rate": 2e-5,
      "num_train_epochs": 2,
      }

  # Create a ClassificationModel
  model = ClassificationModel(
      "bert", 
      "bert-base-multilingual-cased",
      use_cuda=True,
      args=train_args,
  )

  # Train the model
  model.train_model(
      train_dataset, 
  )

  start = i*20000
  end = (i+1)*20000
  additional_data = gab_data_df.iloc[start:end]

  # Make predictions with the model
  predictions, raw_outputs = model.predict(additional_data["text"].to_list())

  additional_data['labels'] = predictions

  train_dataset = train_dataset.append(additional_data)

  train_dataset.to_csv('dataset_' + str(i+2) + '.csv', index=False)

In [22]:
# self training for 5 iterations with 20k additional texts from gab dataset

for i in range(5):

  suffix = str(i+1)
  train_dataset = pd.read_csv('dataset_'+suffix+'.csv')

  training_model(i, train_dataset)
  print("iteration %s completed successfuly" % (i+1))

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

...

Running Epoch 0 of 2:   0%|          | 0/5875 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/5875 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

iteration 5 completed successfuly


# Get predictions for all 5 models

In [24]:
# sanity check

for i in range(6):
  filename = 'dataset_' + str(i+1) + '.csv'
  test = pd.read_csv(filename)
  print('length of %s = %d' % (filename, len(test)))

length of dataset_1.csv = 14000
length of dataset_2.csv = 34000
length of dataset_3.csv = 54000
length of dataset_4.csv = 74000
length of dataset_5.csv = 94000
length of dataset_6.csv = 114000


## model 1

macro F1 score on dev - 0.7950

In [29]:
model1 = ClassificationModel(
    "bert", "iteration_0/checkpoint-1750-epoch-2"
)

dev_df1 = dev_df

# Make predictions with the model
predictions, raw_outputs = model1.predict(dev_df1["text"].to_list())

dev_df1['label_pred'] = predictions
dev_df1 = dev_df1[['rewire_id', 'label_pred']]
dev_df1.head(2)

dev_df1.to_csv('task_a_pred_1_model1.csv', index=False)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

## model 2

macro F1 score on dev - 0.7958

In [34]:
model2 = ClassificationModel(
    "bert", "iteration_1/checkpoint-4250-epoch-2"
)

dev_df2 = dev_df

# Make predictions with the model
predictions, raw_outputs = model2.predict(dev_df2["text"].to_list())

dev_df2['label_pred'] = predictions
dev_df2 = dev_df2[['rewire_id', 'label_pred']]

dev_df2.to_csv('task_a_pred_1_model2.csv', index=False)

dev_df2.head(2)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

,rewire_id,label_pred
0,sexism2022_english-11999,not sexist
1,sexism2022_english-9895,not sexist


## model 3

macro F1 score on dev - 0.7917

In [35]:
model3 = ClassificationModel(
    "bert", "iteration_2/checkpoint-6750-epoch-2"
)

dev_df3 = dev_df

# Make predictions with the model
predictions, raw_outputs = model3.predict(dev_df3["text"].to_list())

dev_df3['label_pred'] = predictions
dev_df3 = dev_df3[['rewire_id', 'label_pred']]

dev_df3.to_csv('task_a_pred_1_model3.csv', index=False)

dev_df3.head(2)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

,rewire_id,label_pred
0,sexism2022_english-11999,not sexist
1,sexism2022_english-9895,not sexist


## model 4

macro F1 score on dev - 0.7842

In [38]:
model4 = ClassificationModel(
    "bert", "iteration_3/checkpoint-9250-epoch-2"
)

dev_df4 = dev_df

# Make predictions with the model
predictions, raw_outputs = model4.predict(dev_df4["text"].to_list())

dev_df4['label_pred'] = predictions
dev_df4 = dev_df4[['rewire_id', 'label_pred']]

dev_df4.to_csv('task_a_pred_1_model4.csv', index=False)

dev_df4.head(2)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

,rewire_id,label_pred
0,sexism2022_english-11999,not sexist
1,sexism2022_english-9895,not sexist


## model 5

macro F1 score on dev - 0.7845

In [39]:
model5 = ClassificationModel(
    "bert", "iteration_4/checkpoint-11750-epoch-2"
)

dev_df5 = dev_df

# Make predictions with the model
predictions, raw_outputs = model5.predict(dev_df5["text"].to_list())

dev_df5['label_pred'] = predictions
dev_df5 = dev_df5[['rewire_id', 'label_pred']]

dev_df5.to_csv('task_a_pred_1_model5.csv', index=False)

dev_df5.head(2)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

,rewire_id,label_pred
0,sexism2022_english-11999,not sexist
1,sexism2022_english-9895,not sexist
